<a href="https://colab.research.google.com/github/luccafiori/GCP_BIGQUERY_NOTEBOOKS/blob/main/Consolidated_Shipments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import pandas as pd

project = 'sz-snop-dev' # Project ID inserted based on the query results selected to explore
location = 'southamerica-east1' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

#job ids in GCP (Queries results)

job_ids = ['bquxjob_49014464_18e1c31b35e', 'bquxjob_65747021_18e1c31a697', 'bquxjob_7c05de01_18e1c318d87']
results_list = [client.get_job(job_id).to_dataframe().assign(job_id=job_id) for job_id in job_ids]

# Concatenate all results into a single dataframe
combined_results = pd.concat(results_list)

# convert time zone
def convert_timezone_aware_to_naive(df):
    for col in combined_results.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns:
        combined_results[col] = combined_results[col].dt.tz_convert(None)
    return combined_results

# Apply the function to your DataFrame
combined_results = convert_timezone_aware_to_naive(combined_results)
#

file_path = 'Consolidated_Shipments.xlsx'
combined_results.to_excel(file_path, index=False)